In [38]:
import pandas as pd
import numpy as np
from datetime import datetime

data = pd.read_csv('sphist.csv')

data.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883
5,2015-11-30,2090.949951,2093.810059,2080.409912,2080.409912,4.245030e+09,2080.409912
6,2015-11-27,2088.820068,2093.290039,2084.129883,2090.110107,1.466840e+09,2090.110107
7,2015-11-25,2089.300049,2093.000000,2086.300049,2088.870117,2.852940e+09,2088.870117
8,2015-11-24,2084.419922,2094.120117,2070.290039,2089.139893,3.884930e+09,2089.139893
9,2015-11-23,2089.409912,2095.610107,2081.389893,2086.590088,3.587980e+09,2086.590088


In [39]:
data['Date'] = pd.to_datetime(data['Date'])

In [40]:
data = data.sort_values('Date', ascending = True)

In [41]:
# generate indicator for average close price from previous 5 days

counter = 0
last_5_avg = []

for i in data['Close']:
    last_5 = 0
    if counter > 5:
        start = counter - 5
        end = counter
        last_5 = data['Close'][start:end].mean()
        last_5_avg.append(last_5)
    else:
        last_5_avg.append(i)
    counter += 1


In [42]:
# generate indicator for average close price from last 30 days

counter = 0
last_30_avg = []

for i in data['Close']:
    last_30 = 0
    if counter > 30:
        start = counter - 30
        end = counter
        last_30 = data['Close'][start:end].mean()
        last_30_avg.append(last_30)
    else:
        last_30_avg.append(i)
    
    counter += 1


In [53]:
# calculate ration of std dev over last 5 days over stdev over last 365 days

counter = 0
ratio_stdev = []

for i in data['Close']:
    ratio = 0
    if counter > 365:
        start_5 = counter - 5
        start_365 = counter - 365
        end - counter
        
        last_5 = data['Close'][start_5:end]
        last_365 = data['Close'][start_365:end]
        
        stdev_5 = np.std(last_5, ddof = 1)
        stdev_365 = np.std(last_365, ddof = 1)
        
        ratio_stdev.append(stdev_5 / stdev_365)
        
    else:
        ratio_stdev.append(i)
    counter += 1   
        
ratio_stdev[364:370]
        
   

[22.049999,
 22.02,
 1.0031865319733433,
 1.0031863470139972,
 1.0031861818484558,
 1.0031860307556248]

In [54]:
# calculate average volume over last 365 days

counter = 0
avg_vol = []

for i in data['Volume']:
    avg = 0
    if counter > 365:
        start = counter - 365
        end = counter
        
        last_365 = data['Volume'][start:end].mean()
        
        avg_vol.append(last_365)
    
    else:
        avg_vol.append(i)
        


In [55]:
# assign lists to columns in the dataset

data['avg_last_5'] = last_5_avg
data['avg_last_30'] = last_30_avg
data['stdev_ratio'] = ratio_stdev
data['avg_vol_365'] = avg_vol

In [56]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16590 entries, 16589 to 0
Data columns (total 11 columns):
Date           16590 non-null datetime64[ns]
Open           16590 non-null float64
High           16590 non-null float64
Low            16590 non-null float64
Close          16590 non-null float64
Volume         16590 non-null float64
Adj Close      16590 non-null float64
avg_last_5     16590 non-null float64
avg_last_30    16590 non-null float64
stdev_ratio    16590 non-null float64
avg_vol_365    16590 non-null float64
dtypes: datetime64[ns](1), float64(10)
memory usage: 1.5 MB
None


In [57]:
# remove rows without enough data - fall before 1951-01-03

stock_data = data[data['Date'] > datetime(year = 1951, month = 1, day = 2)]

stock_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16340 entries, 16339 to 0
Data columns (total 11 columns):
Date           16340 non-null datetime64[ns]
Open           16340 non-null float64
High           16340 non-null float64
Low            16340 non-null float64
Close          16340 non-null float64
Volume         16340 non-null float64
Adj Close      16340 non-null float64
avg_last_5     16340 non-null float64
avg_last_30    16340 non-null float64
stdev_ratio    16340 non-null float64
avg_vol_365    16340 non-null float64
dtypes: datetime64[ns](1), float64(10)
memory usage: 1.5 MB


In [58]:
# generate train and test dfs

split = datetime(year = 2013, month = 1, day = 1)

train = stock_data[stock_data['Date'] < split]
test = stock_data[stock_data['Date'] > split]


In [60]:
# create linear model for the data

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

features = ['avg_last_5', 'avg_last_30', 'stdev_ratio', 'avg_vol_365']
target = ['Close']

lr = LinearRegression()
lr.fit(train[features], train[target])
predictions = lr.predict(test[features])
mae = mean_absolute_error(predictions, test[target])

print(mae)

# mae with avg 5, avg 365, stdev ratio = 16.1483
# mae with avg_col_365 = 16.1028

16.10285877042925
